# Импорт библиотек

In [34]:
!pip install aiogram
!pip install nest_asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
from aiogram import Bot, types
from aiogram.dispatcher import Dispatcher, FSMContext
from aiogram.dispatcher.filters.state import State, StatesGroup
from aiogram.utils import executor
from aiogram.types import ReplyKeyboardMarkup, KeyboardButton, ReplyKeyboardRemove
from aiogram.contrib.fsm_storage.memory import MemoryStorage
import sqlite3 as sq
import nest_asyncio
import requests
import io
from io import BytesIO
from PIL import Image
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam, Adamax

# Импорт моделей

In [36]:
img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)

In [37]:
class_count_xray = 3
class_count_scan = 4
class_count_mri = 15


In [38]:
base_model_xray = tf.keras.applications.xception.Xception(include_top= False, 
                                                               weights= "imagenet", 
                                                               input_shape= img_shape, 
                                                               pooling= 'max')

model_xray = Sequential([
    base_model_xray,
    BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001),
    Dense(256, 
          kernel_regularizer= regularizers.l2(l= 0.016), 
          activity_regularizer= regularizers.l1(0.006),
          bias_regularizer= regularizers.l1(0.006), 
          activation= 'relu'),
    
    Dropout(rate= 0.45, 
            seed= 123),
    
    Dense(class_count_xray, activation= 'softmax')
])

model_xray.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

model_xray.summary()

model_xray.load_weights('/content/drive/MyDrive/Colab Notebooks/ВКР/xray_Xception_model_weights.h5')

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 2048)              20861480  
                                                                 
 batch_normalization_26 (Bat  (None, 2048)             8192      
 chNormalization)                                                
                                                                 
 dense_12 (Dense)            (None, 256)               524544    
                                                                 
 dropout_6 (Dropout)         (None, 256)               0         
                                                                 
 dense_13 (Dense)            (None, 3)                 771       
                                                                 
Total params: 21,394,987
Trainable params: 21,336,363
Non-trainable params: 58,624
_____________________________________

In [39]:
# Получение метки класса из выходного вектора нейронной сети
def get_class_label_xray(result):
    predicted_class = np.argmax(result)
    class_names = ['bacteria', 'nor', 'virus']
    if predicted_class == 1:
      return 'Всё хорошо, нет причин для беспокойства'
    else:
      return 'Подозрение на вид пневмонии: '+ class_names[predicted_class]


In [40]:
base_model_scan = tf.keras.applications.xception.Xception(include_top= False, 
                                                               weights= "imagenet", 
                                                               input_shape= img_shape, 
                                                               pooling= 'max')

model_scan = Sequential([
    base_model_scan,
    BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001),
    Dense(256, 
          kernel_regularizer= regularizers.l2(l= 0.016), 
          activity_regularizer= regularizers.l1(0.006),
          bias_regularizer= regularizers.l1(0.006), 
          activation= 'relu'),
    
    Dropout(rate= 0.45, 
            seed= 123),
    
    Dense(class_count_scan, activation= 'softmax')
])

model_scan.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

model_scan.summary()

model_scan.load_weights('/content/drive/MyDrive/Colab Notebooks/ВКР/scan_Xception_model_weights.h5')

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 2048)              20861480  
                                                                 
 batch_normalization_31 (Bat  (None, 2048)             8192      
 chNormalization)                                                
                                                                 
 dense_14 (Dense)            (None, 256)               524544    
                                                                 
 dropout_7 (Dropout)         (None, 256)               0         
                                                                 
 dense_15 (Dense)            (None, 4)                 1028      
                                                                 
Total params: 21,395,244
Trainable params: 21,336,620
Non-trainable params: 58,624
_____________________________________

In [41]:
# Получение метки класса из выходного вектора нейронной сети
def get_class_label_scan(result):
    predicted_class = np.argmax(result)
    class_names = ['Adenocarcinoma', 'Squamous_cell_carcinoma', 'Large_cell_carcinoma','normal']
    if predicted_class == 3:
      return 'Всё хорошо, нет причин для беспокойства'
    else:
      return 'Подозрение на: '+ class_names[predicted_class]


In [42]:
base_model_mri = tf.keras.applications.efficientnet.EfficientNetB5(include_top= False, 
                                                               weights= "imagenet", 
                                                               input_shape= img_shape, 
                                                               pooling= 'max')

model_mri = Sequential([
    base_model_mri,
    BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001),
    Dense(256, 
          kernel_regularizer= regularizers.l2(l= 0.016), 
          activity_regularizer= regularizers.l1(0.006),
          bias_regularizer= regularizers.l1(0.006), 
          activation= 'relu'),
    
    Dropout(rate= 0.45, 
            seed= 123),
    
    Dense(class_count_mri, activation= 'softmax')
])

model_mri.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

model_mri.summary()

model_mri.load_weights('/content/drive/MyDrive/Colab Notebooks/ВКР/MRI_EfficientNetB5_model_weights.h5')

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb5 (Functional)  (None, 2048)             28513527  
                                                                 
 batch_normalization_32 (Bat  (None, 2048)             8192      
 chNormalization)                                                
                                                                 
 dense_16 (Dense)            (None, 256)               524544    
                                                                 
 dropout_8 (Dropout)         (None, 256)               0         
                                                                 
 dense_17 (Dense)            (None, 15)                3855      
                                                                 
Total params: 29,050,118
Trainable params: 28,873,279
Non-trainable params: 176,839
____________________________________

In [43]:
# Получение метки класса из выходного вектора нейронной сети
def get_class_label_mri(result):
    predicted_class = np.argmax(result)
    class_names = ['Astrocitoma','Carcinoma',
                   'Ependimoma','Ganglioglioma',
                   'Germinoma','Glioblastoma',
                   'Granuloma','Meduloblastoma',
                   'Meningioma','Neurocitoma',
                   'Oligodendroglioma','Papiloma',
                   'Schwannoma','Tuberculoma',
                   '_NORMAL']
    if predicted_class == 14:
      return 'Всё хорошо, нет причин для беспокойства'
    else:
      return 'Подозрение на: '+ class_names[predicted_class]


# Работа бота

In [44]:

storage = MemoryStorage()

bot = Bot(token='')
bot_token  = ''
dp = Dispatcher(bot, storage=storage)


def sql_start():
    global base, cur
    base = sq.connect('/content/drive/MyDrive/Colab Notebooks/ВКР/users.db')
    cur = base.cursor()
    if base:
        print('Data base connected OK!')
    base.execute('CREATE TABLE IF NOT EXISTS users_picture(picture TEXT,research TEXT, id_user PRIMARY KEY)')
    base.commit()


async def sql_add_command(state):
    async with state.proxy() as data:
        cur.execute('INSERT INTO users_picture VALUES(?,?,?)', tuple(data.values()))
        base.commit()


async def sql_update_command(state):
    async with state.proxy() as data:
        cur.execute('UPDATE users_picture SET picture = ? WHERE users_id = ?', tuple(data.values())[::2])
        cur.execute('UPDATE users_picture SET research = ? WHERE users_id = ?', tuple(data.values())[1:])
        base.commit()



def on_startup():
    print("Бот вышел в  онлайн")
    sql_start()


class FSM_picture(StatesGroup):
    picture = State()
    research = State()
    id_user = State()

b0 = KeyboardButton('Сделать запрос')
b1 = KeyboardButton('xray')
b2 = KeyboardButton('scan')
b3 = KeyboardButton('mri')

zero_client = ReplyKeyboardMarkup(resize_keyboard=True,one_time_keyboard=True)
zero_client.add(b0)

first_client = ReplyKeyboardMarkup(resize_keyboard=True)
first_client.add(b1).add(b2).add(b3)

@dp.message_handler(commands=['start'])
async def command_start(message: types.Message):
    await bot.send_message(message.from_user.id, 'Привет, я - бот, который может проанализировать изображение. Выбери пункт меню, который тебе интересен',
                           reply_markup=zero_client)


@dp.message_handler(lambda message: "Начать" in message.text)
async def commands_begin(message: types.Message):
    await bot.send_message(message.from_user.id, 'Привет, я - бот, который может проанализировать изображение. Выбери пункт меню, который тебе интересен',
                           reply_markup=zero_client)
    

@dp.message_handler(lambda message: "Сделать запрос" in message.text)
async def cm_start(message: types.Message):
    await FSM_picture.picture.set()
    await bot.send_message(message.from_user.id, 'Пришли в ответном сообщении изображение, которое нужно проанализировать')


@dp.message_handler(content_types=['photo'],state=FSM_picture.picture)
async def load_picture(message: types.Message, state: FSMContext):
    async with state.proxy() as data:
      data['picture'] = message.photo[-1].file_id
      await message.answer('Изображение получено, теперь укажи тип исседования:\n xray - Проверка на пневмонию \n scan - Проверка на заболевания грудной клетки \n mri - МРТ головного мозга',reply_markup=first_client)  
    await FSM_picture.next()

@dp.message_handler(state=FSM_picture.research)
async def load_research(message: types.Message, state: FSMContext):
    async with state.proxy() as data:
      if message.text == 'xray':
        data['research'] = 'xray'

        file_info = await bot.get_file(data['picture'])
        file_url = f'https://api.telegram.org/file/bot{bot_token}/{file_info.file_path}'
        response = requests.get(file_url)
        img = Image.open(io.BytesIO(response.content))
        img = img.convert('RGB')
        img = img.resize((224, 224)) 
        img_array = np.array(img)
        
        await message.answer('Изображение получено, выполняю обработку')
        
        result = model_xray.predict(np.array([img_array]))
        class_label = get_class_label_xray(result)

        await message.answer(f"Предсказанная метка класса: {class_label}",reply_markup=zero_client)

      if message.text == 'scan':
        data['research'] = 'scan'

        file_info = await bot.get_file(data['picture'])
        file_url = f'https://api.telegram.org/file/bot{bot_token}/{file_info.file_path}'
        response = requests.get(file_url)
        img = Image.open(io.BytesIO(response.content))
        img = img.convert('RGB')
        img = img.resize((224, 224)) 
        img_array = np.array(img)
        
        await message.answer('Изображение получено, выполняю обработку')
        
        result = model_scan.predict(np.array([img_array]))
        class_label = get_class_label_scan(result)

        await message.answer(f"Предсказанная метка класса: {class_label}",reply_markup=zero_client)

      if message.text == 'mri':
        data['research'] = 'mri'

        file_info = await bot.get_file(data['picture'])
        file_url = f'https://api.telegram.org/file/bot{bot_token}/{file_info.file_path}'
        response = requests.get(file_url)
        img = Image.open(io.BytesIO(response.content))
        img = img.convert('RGB')
        img = img.resize((224, 224)) 
        img_array = np.array(img)
        
        await message.answer('Изображение получено, выполняю обработку')
        
        result = model_mri.predict(np.array([img_array]))
        class_label = get_class_label_mri(result)

        await message.answer(f"Предсказанная метка класса: {class_label}",reply_markup=zero_client)
      else:
        data['research'] = 'none'

    async with state.proxy() as data:
        data['id_user'] = message.from_user.id
    for ret in cur.execute('SELECT * FROM users_picture').fetchall():
        if message.from_user.id == ret[2]:
            await sql_update_command(state)
        else:
            await sql_add_command(state)
    await state.finish()

@dp.message_handler()
async def echo_send(message: types.Message):
    await message.answer('Простите, но я не понимаю ваш запрос. Не могли бы вы сказать иначе?')



# Запуск бота

In [46]:
nest_asyncio.apply()
executor.start_polling(dp, skip_updates=True, on_startup=on_startup())

Бот вышел в  онлайн
Data base connected OK!


1/1 [==============================] - 0s 227ms/step
